In [69]:
def pwr(x: int, power: int, modulo: int) -> int:
    '''
        calc x ^ power % modulo
    '''
    if power == 0:
        return 1
    if power % 2 == 1:
        res = pwr(x, power - 1, modulo) * x
        return res % modulo
    res = pwr(x, power // 2, modulo)
    res *= res
    return res % modulo


def gcd_ext(x: int, y: int):
    '''
        calc greatest common divisor, also return a, b such that x * a + b * y = gcd(x,y)
    '''
    if x == 0:
        return y, 0, 1
    d, a1, b1 = gcd_ext(y % x, x)
    a = b1 - (y // x) * a1
    b = a1
    return d, a, b

def inverse(x: int, modulo: int):
    '''
        find y such that x * y % modulo = 1
        make sure that gcd(x, modulo) = 1
    '''
    d, a, b = gcd_ext(x, modulo)
    assert d == 1, d
    a %= modulo
    assert 0 <= a < modulo, a
    return a

In [48]:
gcd_ext(12, 18)

(6, -1, 1)

In [49]:
pwr(2, 3, 12)

8

In [50]:
assert (inverse(7, 114) * 7) % 114 == 1

In [95]:
MODULO = 57896044618658097711785492504343953926634992332820282019728792003956564819937  # <- is prime

In [73]:
cipher(3), cipher(4)

(482873, 1003436)

In [74]:
decipher(cipher(3)), decipher(cipher(4))

(3, 4)

In [100]:
class PowerCipher:
    '''
        Cipher is based on property that it's too hard to find power in equasion: (a ^ power) % modulo = b
        when a,b,modulo are known
        so power is kind of private key
    '''
    def __init__(self, power: int, prime_modulo: int):
        self._power = power
        self._prime_modulo = prime_modulo  # here we won't check if modulo is prime, make it sure yourself
        # it's necessary property to find inverse power that gcd(power, prime_modulo - 1) = 1, it will be checked in next line
        self._inv_power = inverse(power, prime_modulo - 1)  # since Euler_function(prime_modulo) = prime_modulo - 1
        
    def cipher(self, msg: int):
        assert 1 < msg < self._prime_modulo
        return pwr(msg, self._power, self._prime_modulo)
    
    def decipher(self, msg: int):
        assert 1 < msg < self._prime_modulo
        return pwr(msg, self._inv_power, self._prime_modulo)

In [103]:
private1 = 1003
private2 = 725

c1 = PowerCipher(private1, MODULO)
c2 = PowerCipher(private2, MODULO)

for i in range(2, 2 + 52):
    assert i == c1.decipher(c1.cipher(i))
    assert i == c2.decipher(c2.cipher(i))
    assert c1.cipher(c2.cipher(i)) == c2.cipher(c1.cipher(i))

In [107]:
13159035226703211890654003961158071376254143192955391095350192585677108936704 < 2 ** 255

True

In [104]:
for i in range(52):
    p1 = c1.cipher(i + 2)
    p21 = c2.cipher(p1)
    pm121 = c1.decipher(p21)
    print(i, p1, p21, pm121)

0 13159035226703211890654003961158071376254143192955391095350192585677108936704 51013047440355121923165292692991090978079692437179959503745594864127759167634 50602556146741728208349612302003642729229093339963847957696424706048
1 37878010719124385700487666027363824721736664517050548994860851193330156557245 16420545267391948259698966724866868098626549749127394930985664133419648495024 31218259840381562386561664312192665239331968848401774299429617148806591534914
2 34821281519648916360393740224329319049054724583846555594474891520496478715935 920308258963906003294894349273497882273524302583022434353556775854667457547 1371063945196131339848710285958978166078362773706306138144768
3 31265957008351574199682529208389467908610320791293948666160179728842439664637 51798669168145157971653425402731356810004990851148249745953552169296182915981 47973127203639628266168456194645044912891966240061260446654987028540843575624
4 12587466323649879691663785652723033139754191211817995582480202019574199966296 538